### Задание 3 - Индекс | Булев поиск

In [4]:
import nltk
nltk.download("stopwords")
nltk.download('wordnet') 
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import os
from bs4 import BeautifulSoup

In [11]:
# словарь (номер документа -> контент (заголовок + содержимое статьи))
content = {}
i = 1
for file_ in os.listdir('links'):
  if (file_.__contains__('.html')):
    path_ = os.path.join('links', file_)
    file_text = open(path_).read()
  
    soup = BeautifulSoup(file_text)
    
    page_content = []
    for text in soup.find('h1', {'class': 'tm-article-snippet__title tm-article-snippet__title_h1'}):
      title = text.get_text()
      page_content.append(title)

    data = soup.find('div', {'xmlns': 'http://www.w3.org/1999/xhtml'})
    for p in data.find_all('p'):
       text = p.get_text()
       page_content.append(text)
    
    content[i] = page_content
    i += 1

In [12]:
# пример текста 3 документа
content[3]

['Binance и Kraken отказались блокировать россиян: «Это противоречит природе крипты»']

In [13]:
import string
from nltk.corpus import stopwords
import re

In [14]:
def tokenize(content):
    result = []
    tokens = set() 
    for c in content:
      tokens_ = nltk.word_tokenize(c)
      tokens_ = [i.lower() for i in tokens_]
      tokens_ = [i for i in tokens_ if ( i not in string.punctuation )]
      tokens_ = filter(None, [re.sub(r"[^a-zа-я-]+", r"", i) for i in tokens_])

      stop_words = stopwords.words('russian')
      stop_words.extend(['что', 'это', 'так', 'вот', 'как', 'в', 'к',
                         'на', 'о', 'при', 'из-за', 'за', 'ао', 'но', 'х',
                         'хотя', 'среди', 'помимо', 'с'])
      tokens_ = [i for i in tokens_ if ( i not in stop_words )]

      stop_words2 = stopwords.words('english')
      stop_words2.extend(['the', 'e', 'a', 'd', 'b', 'x', 'c'])
      tokens_ = [i for i in tokens_ if ( i not in stop_words2 )]

      tokens_ = [i.replace("«", "").replace("»", "") for i in tokens_]

      for t_ in tokens_:
        tokens.add(t_)
      result += [tokens]

    return result

In [15]:
index={}
for i in range(1, 101):
    data = []
    for c in tokenize(content[i]):
        data += c
    data = list(dict.fromkeys(data))
    for token in data:
        if token in index.keys():
            index[token].append(i)
        else:
            index[token] = [i]

In [16]:
with open('inverted_index.txt', "w", encoding="utf-8") as f:
    for key in index:
        f.write('{\"count\":' + str(len(index[key])) + ',\"inverted_array\":' + str(index[key]) + ',\"word\":\"' + key + '\"}\n')

In [ ]:
from google.colab import files
files.download('inverted_index.txt')

### Реализация булева поиска

In [116]:
def preprocess_for_search(content):
    result = []
    tokens = set() 
    
    tokens_ = nltk.word_tokenize(content)
    tokens_ = [i.lower() for i in tokens_]
    tokens_ = [i for i in tokens_ if ( i not in string.punctuation )]
    tokens_ = filter(None, [re.sub(r"[^a-zа-я-]+", r"", i) for i in tokens_])

    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'как', 'в', 'к',
                         'на', 'о', 'при', 'из-за', 'за', 'ао', 'но', 'х',
                         'хотя', 'среди', 'помимо', 'с'])
    tokens_ = [i for i in tokens_ if ( i not in stop_words )]

    stop_words2 = stopwords.words('english')
    stop_words2.extend(['the', 'e', 'a', 'd', 'b', 'x', 'c'])
    tokens_ = [i for i in tokens_ if ( i not in stop_words2 )]

    tokens_ = [i.replace("«", "").replace("»", "") for i in tokens_]

    for t_ in tokens_:
      tokens.add(t_)
    result += [tokens]

    return result

In [97]:
request = "крипты AND блокировать OR ai"
tokens_search = []
for t in preprocess_for_search(request):
    tokens_search += t

In [98]:
tokens_search

['блокировать', 'крипты', 'ai']

In [99]:
# поиск пересечения нужных слов и слов из созданного индекса
def find_token_in_index(token):
  search = {}
  if token in index.keys():
        search[token] = index[token]
  else:
        search[token] = []
  return search

In [101]:
request = request.split(' OR ')

In [125]:
result = {}
docs = []
for req in request:
  ands = req.split(' AND ')
  temp_searched_docs = []
  temp_result = []
  for token in ands:
    d = find_token_in_index(token)
    if d[token]: 
      temp_searched_docs.append(d[token])
    else: 
      temp_searched_docs = []
      break
  if temp_searched_docs:
    temp_result = list(set.intersection(*map(set, temp_searched_docs)))
    if temp_result:
      result[req] = temp_result
      docs.append(temp_result)


In [127]:
result

{'ai': [10], 'крипты AND блокировать': [3]}

In [128]:
# Документы - результат запроса
docs

[[3], [10]]

### Прошлый вариант булева поиска


In [118]:
# Текст - Тело упадет со скоростью 2,6 тыс. м/c (из 3 документа)
text = "Тело упадет со скоростью 2,6 тыс. м/c"
tokens_search_ = []
for t in preprocess_for_search(text):
    tokens_search_ += t

In [119]:
tokens_search_

['мc', 'скоростью', 'тыс', 'упадет', 'тело']

In [120]:
# поиск пересечения нужных слов и слов из созданного индекса
search = {}
for token in tokens_search_:
    if token in index.keys():
        search[token] = index[token]
    else:
        search[token] = []

In [121]:
search

{'мc': [8, 91],
 'скоростью': [8, 79, 91],
 'тело': [2, 91],
 'тыс': [5, 7, 17, 68, 91, 99],
 'упадет': [91]}

In [122]:
def bool_search(search_type, search):
  i = 1
  document = []
  for token in search:
      if i == 1:
        document = search[token]
      else:
        if search_type == 'AND':
           # все токены должны быть в документе
          document = list(set(document) & set(search[token]))
        if search_type == 'OR':
           # достаточно хотя бы одного токена в документе
          document = list(set(document) | set(search[token]))
      i += 1
  print(document)

In [123]:
bool_search('AND', search)

[91]


In [124]:
bool_search('OR', search)

[2, 99, 68, 5, 7, 8, 79, 17, 91]
